# Base de datos: Trabajo dos 

## Librerias usadas

In [3]:
import numpy as np
import pandas as pd
import re
from collections import Counter  

# Codigo empleado 

## 1. Dependencias funcionales completas

In [4]:



class dependencias:   # clase que almacena funciones para la dependencias 

    def pegar(x,y):       # retorna una lista 
        xy = []           # donde se almacenara las columnas pegadas 
        for i in range(len(x)):    # itera la lista x
            xy.append(str(x[i])+str(y[i]))   # agrega y une cada elemento de x,y
        return(xy)   # retorno 
    
    def minimo(lista):    # regresa la longitud minima de de los elementos de una lista
        length = 4        # parametro maximo
        for j in lista:   # itera la lista
            if len(j) < length:   # condicion de minimo
                length = len(j)   # nuevo valor de length si se cumple la condicion
        return length             # retorno


    def dependencia(determina, determinado):    # retrona True o False dependiendo si entre las dos determina -> determinado
        a = dependencias.pegar(determina, determinado)  # usa la funcion pegar 
        b = determina    # nueva asignacion 

        a_set = set(a)    # elemenos diferentes d a 
        b_set = set(b)    #elementos diferentes de b

        cumple = len(a_set) == len(b_set)      # si True es porque dtermina -> determinado 

        return(cumple)   # retorno 


    def combinaciones(datos):    # combinaciones de dos atributos 
        df = pd.DataFrame()      # define data frame  (df)
        for i in datos.columns:   # itera las columas de datos
            for j in datos.columns:  # itera las columas de datos
                if(i>j):    # si no son la misma culuma : 
                    df[i+","+j] =dependencias.pegar(datos[i], datos[j])   # se unen 
        return(df)  # retorno 


    def combinaciones2(df_original, df_alterado):   # similar a la funcion anterior solo que se unen un df con el reusltado del anterior 

        df = pd.DataFrame()

        for i in df_original.columns:
            for j in df_alterado.columns:
                if (bool(re.search(i, j)) == False):
                    df[i+","+j] = dependencias.pegar(df_original[i], df_alterado[j])

        return(df)


    def todasDependencias1(datos):           # dependencias de 1 a 1:  una varable determina a otra 
    
        df = pd.DataFrame()
        x = []          # donde se almacenan las que detrminan
        y = []          # donde se almacena la determinada    

        for i in datos.columns:
            for j in datos.columns:
                if (i != j and dependencias.dependencia(datos[i],datos[j])):  # condicon 
                    #print("{",i,"}","->", j)
                    x.append(i); y.append(j)   # agregar a a y y respectivamente 
        df["x"] = x; df["y"] = y   # unor x,y a df 

        return(df)  # retorno de un df



    def todasDependencias2(df_original, df_alterado):   # dependencias cuando dos variables determinan a una 


        df = pd.DataFrame()
        x = []          # donde se almacenan las que detrminan
        y = []          # donde se almacena la determinada

        for i in df_original.columns:  # itera los datos originales  
            for j in df_alterado.columns:    # itera los datos combinados  
                if (bool(re.search(i, j)) == False and dependencias.dependencia(df_alterado[j],df_original[i])):   # si comple con la condicion j->i
                    #print("{",j,"}","->", i)
                    x.append(j); y.append(i)

        df["x"] = x; df["y"] = y
        return(df)  # retorno de un df


    def todas(datos):   # funcion que usa las funcioenes anteriores 
        df2 =dependencias.combinaciones(datos)   
        df3 =dependencias.combinaciones2(datos,df2)
        d1 = dependencias.todasDependencias1(datos)
        d2 = dependencias.todasDependencias2(datos, df2)   
        d3 = dependencias.todasDependencias2(datos, df3)  
        alls = pd.merge(pd.merge(d1, d2, how='outer'), d3, how='outer')   # unea d1 y d2 
        return(alls)
    
    
    
    def compl(datos):  # dependecias completas: 
    
    
        #d = cambiar_nombre(datos)
        d = datos

        dep  = dependencias.todas(d)   # regresa un df de todas las dependencias (no completas)

        for i in range(len(dep[["x"]])):
            dep["x"][i] = "".join(sorted(re.split("\W",  dep["x"][i] )))   # almacena en un string las que determinan 
                   # y elimina las dpendencias no completas 

        cp = pd.DataFrame(dep.groupby(['x', 'y']).apply(list).reset_index())  # agrupa 

        cp = cp.groupby(['y'])['x'].apply(list).reset_index()  #a agrupa para no tener filas repetidas 



        new_x = []      # dfine una lista

        for i in cp.index:   # recooree las filas de cp
            minm = dependencias.minimo(cp["x"][i])     # aplica la funcion minimo 
            new_xi = cp["x"][i]     
    
            for i in new_xi:    # itera el nuevo x_i 
                if len(i) == 3:
                    new_xi.remove(i)     # limina 
                
                else:
                    for i in new_xi:   # itera 
                        if len(i) == minm:  # condicion para que entre 
                            for j in new_xi:   # 
                                if len(j) > minm and bool(re.search(i, j)):
                                    new_xi.remove(j)   # elimina para dejar lo completas 
            new_x.append(new_xi)



        cp["x"] = new_x    # eliina las no completas 

        return cp[sorted(cp.columns)]
    
    def completas(datos):  # dependencias completas (las muestra mas organizadas)
        
        dd = dependencias.compl(datos)   # alica la funcion anterior 
        a = dd["x"]  # separa las variables 
        b = dd["y"] 
        
        aa = []   # define lista para alamcenar 
        bb = []

        for i in range(len(a)):    # itera a 
            if len(a[i])>1:   # condicion de filtro 
                for j in range(len(a[i])): # itera a os elementos de ai
                    aa.append(a[i][j]) # agraga 
                    bb.append(b[i]) # agrega 
            else:
                aa.append(a[i][0])  # agrega a
                bb.append(b[i])
                
        dff = pd.DataFrame()  # define df

        dff["X"] = aa   # alameca en el df
        dff["Y"] = bb
        
        
        return dff  # detorna dff 
   

# 2. Clave primaria

In [5]:



class clavePrimaria:     # define la clase
    
    def _clave1(datos):     # define funcion 
        
        clave = []   # para alacnar claves 
        
        for i in datos.columns:   # itera las columnas 
            if(len(datos[i].value_counts()) == len(datos.index)):   # si la longitud de los indices es igual a el numro diferentes de valores de la columna i-->> la columna i es clave primaria 
                clave.append(i)  # se agrega 
        return(clave)  # rerona la clave primaria 
    
        
    def _clave2(datos):   # para clave primaria en parejas 
        clave1 = []
        clave2 = []
        for i in datos.columns:  # iterar columnas
            for j in datos.columns:
                if (j > i):   # solo entra si las columnas son dferentes 
                    v = []  # se alacenaran las claves 
                    sj = [str(int) for int in datos[j]]   # nombre de las columnas 
                    si = [str(int) for int in datos[i]]   # 
                    for k in range(len(si)):
                        v.append(si[k]+sj[k])   # pega las dos columnas 
                    if(len(Counter(v))==len(si)):    # condicon para ser clave primaria 
                        clave1.append(i)   # se agregan 
                        clave2.append(j)
        return([clave1,clave2])  # retorna clave primaria compuesta 
    
    def _clave3(datos):   # similar a la funcion anteriro agregando una jerarquia adicional 
        clave1 = []; clave2 = [];clave3 = []
        for i in datos.columns:
            for j in datos.columns:
                for k in datos.columns:
                    if (k > j > i):
                        v = []
                        si = [str(int) for int in datos[i]]
                        sj = [str(int) for int in datos[j]]
                        sk = [str(int) for int in datos[k]]
                        for z in range(len(si)):
                            v.append(si[z]+sj[z]+sk[z])
                        if(len(Counter(v))==len(si)):
                            clave1.append(i)
                            clave2.append(j)
                            clave3.append(k)
        return([clave1,clave2,clave3])
    
    def _clave4(datos):     # para el caso de solo existir una clave primaria. 
        # sima dinamica de lo anterior solo con jerarqia adicional 
        clave1 = []; clave2 = []
        clave3 = [];clave4=[]
        for i in datos.columns:
            for j in datos.columns:
                for k in datos.columns:
                    for l in datos.columns:
                        if (l> k > j > i):
                            v = []
                            si = [str(int) for int in datos[i]]
                            sj = [str(int) for int in datos[j]]
                            sk = [str(int) for int in datos[k]]
                            sl = [str(int) for int in datos[l]]
                            for z in range(len(si)):
                                v.append(si[z]+sj[z]+sk[z] + sl[z])
                            if(len(Counter(v))==len(si)):
                                clave1.append(i);clave2.append(j)
                                clave3.append(k);clave4.append(l)
        return([clave1,clave2,clave3,clave4])
    
    
    def algoritmo(datos):   # resuelve las condiciones para la eixstencia de las claves primarias 
        
        #   en cada caso aplica la funcion y rtorna la o las claves primarias (de un lementos o compuestas )
        
        
        if(len(clavePrimaria._clave1(datos)) != 0):  # si la clve es de a una variable 
            
            clave = np.array(clavePrimaria._clave1(datos)).T.tolist()
            return(clave)
        
        elif(np.array(clavePrimaria._clave2(datos)).shape[1]  != 0): # si la clve es de a dos variable s
            
            clave = np.array(clavePrimaria._clave2(datos)).T.tolist()
            return(clave)
        
        elif(np.array(clavePrimaria._clave3(datos)).shape[1]  != 0): # si la clve es de a 3 variable s
            
            clave = np.array(clavePrimaria._clave3(datos)).T.tolist()
            return(clave)
        
        elif(np.array(clavePrimaria._clave4(datos)).shape[1]  != 0): # si la clve es de a 4 variable s
            
            clave = np.array(clavePrimaria._clave4(datos)).T.tolist()
            return(clave)
        
        else:
            
            return('la tabla no contine clave primaria')

    def mostrarClave(datos):    # muestra la clave mas ordenada 
        clave = clavePrimaria.algoritmo(datos)    
        if len(np.array(clave).shape) == 1:  # reotrino si no es compuesta 
            for i in range(len(clave)):
                print('{',clave[i],'}')    # retornar si es clave compuesta 
        else:
            for i in range(len(clave)):
                print('{'+(', '.join([str(elem) for elem in clave[i]]))+'}')   # forma para reornar de forma estetica 





## 3. Cierre Mínimo

In [6]:


class Cierre:   # define clase 
    
    def cierre_minimo(datos):     # funcion de cieere minimo 
    
    
        d = dependencias.completas(datos)    # odas las dependencias completas 
        d = d.groupby(['X'])['Y'].apply(list).reset_index()  # agrupa las dependencias 

        a_C = "888"  # condiciones para no roper el while 
        a_B = "73"

        while a_C != a_B:    
    
    
            a_A = d["Y"].copy()    # copias de la determinadas 
            a_B = ''.join([str(elem) for elem in a_A])   # los atributos que determinana (X)
            for i in range(len(d["X"])):    # iterar las filas de d 
                for j in range(len(d["Y"][i])):    # iterar las determinadas 
                    if (Cierre.strange(i, d["Y"][i][j], d)) and len(d["Y"][i])>1:   # condicion 
                        d["Y"][i].pop(j)  # eliminar si es estraño 
                        break
                
            a_C = ''.join([str(elem) for elem in d["Y"]])  # informacion util para romper el while 
        
        
        return d
    
    
    def cierreMinimo(datos):  # organiza el cierre minimo 
        
        dd = Cierre.cierre_minimo(datos)  # apica el cieerre minimo 

        a = dd["X"]  # define a a y b 
        b = dd["Y"]
        
        aa = []  # crea listas vacias 
        bb = []

        # para no mostrar dpendecnias agrupadas 
        
        for i in range(len(a)):   # itera a a 
            if len(b[i])>1:   # cpndicon
                for j in range(len(b[i])):  # tera a bi si bi tiene mas de un elemento 
                    aa.append(a[i])       
                    bb.append(b[i][j])
            else:
                aa.append(a[i])   # codicio su len(bi) == 1
                bb.append(b[i][0])

        dff = pd.DataFrame() 

        dff["X"] = aa
        dff["Y"] = bb
        
        return dff



    def strange(h, estra, df):           # entrada del indice de la fila y un posible extraño
    
    
        copia = df["Y"][h].copy()     # hacer una copia de generados   
   
        cierre = df["Y"][h].copy()            # cierre 
        cierre.remove(estra)
        ciere_i = 0
        cierre_i1 = -1
        while (ciere_i != cierre_i1):
            ciere_i = len(cierre)
            for i in range(len(df["X"])):     # iterar todas las filas 
    
                k = ''.join([str(elem) for elem in cierre])        # volver el cierre en un string 
                l = len(df["X"][i])                                 # longitud
                c = 0
                for j in range(l):
                    c = c + bool(re.search(df["X"][i][j], k))
                if c == l:
                    for s in df["Y"][i]:
                        if False == bool(re.search(s, k)):
                            cierre.append(s)  
                cierre_i1 = len(cierre)

            
        n = 0
        long = len(copia)    # longitud de la copia 
        cierr = ''.join([str(elem) for elem in cierre])  # cierre 
        for i in range(long):    
            if bool(re.search(copia[i], cierr)):   
                n = n + 1
            
            

        return (n == long)    # condicio  para que exista un estraño
    
    
  

## 4. Algoritmo 3NF

In [7]:
    
class TreeNF:   # clase 

    def pegar(dd):    # pega dos listas ( hace la misma funcion de la funcion pegar de la clase dependencia)
        x = dd["X"]
        y = dd["Y"]  # define a x y y 
        xy = []   # donde se almacenaran los elementos pegados 
        for i in dd.index:
            xy.append(str(x[i])+str(y[i]))   # se une cada eleemento 
        return(xy)

    def reduntante(datos):   # si hay redundancia 
    
        cm = Cierre.cierreMinimo(datos)   # aplica el cierre minimo 
        cm_p = TreeNF.pegar(cm)   # pega los elementos de cm  (solo hay dos) 

        for i in range(len(cm_p)):     # itera a cm_p
            for j in range(len(cm_p)):   # itera a cm_p 
                if i != j and bool(re.search("".join(sorted(cm_p[i])), "".join(sorted(cm_p[j])))):  # condicion para la existencia de redundancia 
                    cm = cm.drop(labels=i, axis=0)   # limina la redundancia 

 
                break    # rompe cuando ya no existe redundancia 

           
        return cm  # retorna el cierre no redundante 
    
    
    def treeNF(datos, copy):   # ingresa los datos y una copia d ellos 

        dic = {}  # crea un diccionario para alamecnar los atributos 
        nombre = copy.columns.tolist()   # copia 
        new_nombre = ["w","x","y","z"]    # nombres nuevos 
        nn = nombre.copy()    # copia de nombres 
        cpp = clavePrimaria.algoritmo(datos)   # aplica clave primaria 
        for i in range(len(nn)):   # asigna nombres nuevos 
            dic[new_nombre[i]] = nn[i]
            

        if np.array(cpp).shape[0] == 1:  # si solo existe una clave primaria 
    
            r = TreeNF.reduntante(datos)   # elimina los redundanres 
    
    
            if len(set(TreeNF.reduntante(datos)["X"])) ==1:   # condicion para cuando la clave primaria define a todas y no hay dependencias entre ellas 
                da = datos.rename(columns=dic)  # renombre los datos 
                print(da)  # retorna los datos 
        
            else:  # condicion si hay dependencias entre las los atributos 
        
                for i in r.index:  # itera el indice de las dependencias no reduntdantes 
                    x = []   # lita vacia 
                    for j in "".join((r.iloc[i].values)):   # itera 
                        x.append(j)    # almacena  
                        da =datos[x].groupby(x).apply(list).reset_index()[x]     # agupa para eliminar las parejas 
                    print(da.rename(columns=dic))   # renobra con los atributos  originales 
                    print("-----------------------------")
        
        else:   # condicion si existe mas de una clave primaria 
            
            print("El algoritmo de descomposición no aplica")


# 5. Todo

In [23]:


def BaseDatos(datos):    # donde se genera toda la entrega 
    
    print("============================================")
    print("0. Datos")
    print("--------"); print()
    print(datos)
    copy = datos.copy()   # copia de los datos 

    nombre = datos.columns.tolist()  # nombre de los datos 
    nn = nombre.copy()  # copia de los nombres 

    df = pd.DataFrame()  # crea un df 
    df["nombres"] = nn  # asiga al def los nombre 
    new_nombre = ["w","x","y","z"]  # posible snombre s
    new = []  # lista vacia 
    for i in range(len(nombre)):  # itera los nombres 
        new.append(new_nombre[i])   # asiga nuevo nombre 

    datos.columns = new  # asigna nuevo nombre 
    df["new"] = new   # agrega variable al df 
    df = df.set_index("new")  # new como indice de df 


    DP = dependencias.completas(datos)   # aplica la dependencia 
    
    print("============================================")
    print("1. Dependencias funcionales completas") # print 
    print("----------------------------------"); print()
    for i in DP.index:    # itera DP 
        if len(DP["X"][i]) == 1:   # condicion 
            print('{',df["nombres"][DP["X"][i]],'}',"->", '{',df["nombres"][DP["Y"][i]],'}')  # muetsra ordenada las dependencias 
        else:
            print('{',', '.join([str(df["nombres"][elem]) for elem in  DP["X"][i]]),'}' ,"->", '{',DP["Y"][i],'}')   # muestra ordenada las dpendencias 
    print()
    print("============================================")
    print("2. Clave primaria ")
    print("------------------"); print()


    clavePrimaria.mostrarClave(copy); print()  # aplica la clave primaria de copia 
    print("============================================")
    print("3. Cierre Minimo ")
    print("--------------"); print()


    DP = Cierre.cierreMinimo(datos)  # aplica cierre minimo 
    for i in DP.index:  # itera cierre minimo 
        if len(DP["X"][i]) == 1: # condicion 
            print('{',df["nombres"][DP["X"][i]],'}',"->", '{',df["nombres"][DP["Y"][i]],'}')  # imprime de forma ordenada 
        else:
            print('{',', '.join([str(df["nombres"][elem]) for elem in  DP["X"][i]]),'}' ,"->", '{',DP["Y"][i],'}')  # inrime de fora ordenada 

    print()
    print("============================================")
    print("4. Algoritmo 3NF")
    print("----------------")

    TreeNF.treeNF(datos, copy)   # imprime el argorimo 3NF 
    



# Ejemplos


**Nota**: Los datos se pueden cargar usando la libreria `pandas`, el argumento que recibe la función `entrega` es de tipo `pd.DataFrame()`

In [33]:
datos1 = pd.read_csv('../datos/ejemplo1.txt', sep = ' ')  
datos2 = pd.read_csv('../datos/ejemplo2.txt', sep = ' ')
datos3 = pd.read_csv('../datos/ejemplo3.txt', sep = ' ')
datos4= pd.read_csv('../datos/ejemplo4.txt', sep = ',')
datos5= pd.read_csv('../datos/ejemplo5.txt', sep = ',')
datos6= pd.read_csv('../datos/ejemplo6.txt', sep = ',')
datos7= pd.read_csv('../datos/ejemplo7.txt', sep = ',')
datos8= pd.read_csv('../datos/ejemplo8.txt', sep = ',')

## Ejemplo 1

In [34]:
datos = datos1
BaseDatos(datos)

0. Datos
--------

   cédula  nombre  salario
0     256    Lina      100
1     943  Carlos      110
2     119    Juan       50
3     221    Juan       60
4     333   María       50
5     888    Juan       50
1. Dependencias funcionales completas
----------------------------------

{ cédula } -> { nombre }
{ cédula } -> { salario }

2. Clave primaria 
------------------

{ cédula }

3. Cierre Minimo 
--------------

{ cédula } -> { nombre }
{ cédula } -> { salario }

4. Algoritmo 3NF
----------------
   cédula  nombre  salario
0     256    Lina      100
1     943  Carlos      110
2     119    Juan       50
3     221    Juan       60
4     333   María       50
5     888    Juan       50


## Ejemplo 2

In [35]:
datos = datos2
BaseDatos(datos)

0. Datos
--------

   nit pdto  color  cantidad
0   10   p1   azul        50
1   10   p2   azul        50
2   10   p3  verde        10
3   10   p4   azul        80
4   10   p5   rojo        50
5   20   p1   azul        60
6   20   p3  verde        50
7   30   p1   azul        50
1. Dependencias funcionales completas
----------------------------------

{ pdto } -> { color }
{ nit, pdto } -> { z }

2. Clave primaria 
------------------

{nit, pdto}

3. Cierre Minimo 
--------------

{ nit, pdto } -> { z }
{ pdto } -> { color }

4. Algoritmo 3NF
----------------
   nit pdto  cantidad
0   10   p1        50
1   10   p2        50
2   10   p3        10
3   10   p4        80
4   10   p5        50
5   20   p1        60
6   20   p3        50
7   30   p1        50
-----------------------------
  pdto  color
0   p1   azul
1   p2   azul
2   p3  verde
3   p4   azul
4   p5   rojo
-----------------------------


## Ejemplo 3

In [36]:
datos = datos3
BaseDatos(datos)

0. Datos
--------

   código     país   idioma
0       1  Francia  Francés
1       3  Francia  Francés
2       6      USA   Inglés
3       8       UK   Inglés
4       9       UK   Inglés
1. Dependencias funcionales completas
----------------------------------

{ código } -> { país }
{ código } -> { idioma }
{ país } -> { idioma }

2. Clave primaria 
------------------

{ código }

3. Cierre Minimo 
--------------

{ código } -> { país }
{ país } -> { idioma }

4. Algoritmo 3NF
----------------
   código     país
0       1  Francia
1       3  Francia
2       6      USA
3       8       UK
4       9       UK
-----------------------------
      país   idioma
0  Francia  Francés
1       UK   Inglés
2      USA   Inglés
-----------------------------


## Ejemplo 4

In [37]:
datos = datos4
BaseDatos(datos)

0. Datos
--------

   x1  x2  x3
0   1   2   5
1   2   2   6
2   3   3   7
3   4   3   5
4   1   4   6
5   2   4   7
6   3   5   8
7   4   5   9
1. Dependencias funcionales completas
----------------------------------

{ x2, x3 } -> { w }
{ x1, x3 } -> { x }
{ x1, x2 } -> { y }

2. Clave primaria 
------------------

{x1, x2}
{x1, x3}
{x2, x3}

3. Cierre Minimo 
--------------

{ x1, x2 } -> { y }
{ x1, x3 } -> { x }
{ x2, x3 } -> { w }

4. Algoritmo 3NF
----------------
El algoritmo de descomposición no aplica


## Ejemplo 5

In [38]:
datos = datos5
BaseDatos(datos)

0. Datos
--------

   a  b  c  d
0  1  2  3  4
1  2  3  4  5
2  3  4  5  5
3  4  5  6  7
1. Dependencias funcionales completas
----------------------------------

{ b } -> { a }
{ c } -> { a }
{ a } -> { b }
{ c } -> { b }
{ a } -> { c }
{ b } -> { c }
{ a } -> { d }
{ b } -> { d }
{ c } -> { d }

2. Clave primaria 
------------------

{ a }
{ b }
{ c }

3. Cierre Minimo 
--------------

{ a } -> { c }
{ b } -> { c }
{ c } -> { b }

4. Algoritmo 3NF
----------------
El algoritmo de descomposición no aplica


## Ejemplo 6

In [39]:
datos = datos6
BaseDatos(datos)

0. Datos
--------

   w  x  y   z
0  b  2  5  11
1  c  3  6  12
2  a  1  4  10
3  a  1  4  30
4  b  2  9  11
5  c  3  5  11
1. Dependencias funcionales completas
----------------------------------

{ x } -> { w }
{ w } -> { x }

2. Clave primaria 
------------------

{w, y, z}
{x, y, z}

3. Cierre Minimo 
--------------

{ w } -> { x }
{ x } -> { w }

4. Algoritmo 3NF
----------------
El algoritmo de descomposición no aplica


## Ejemplo 7

In [40]:
datos = datos7
BaseDatos(datos)

0. Datos
--------

   a   b    c
0  1  11  101
1  2  12  102
2  3  13  103
3  4  14  104
4  5  15  105
1. Dependencias funcionales completas
----------------------------------

{ b } -> { a }
{ c } -> { a }
{ a } -> { b }
{ c } -> { b }
{ a } -> { c }
{ b } -> { c }

2. Clave primaria 
------------------

{ a }
{ b }
{ c }

3. Cierre Minimo 
--------------

{ a } -> { c }
{ b } -> { c }
{ c } -> { b }

4. Algoritmo 3NF
----------------
El algoritmo de descomposición no aplica


## Ejemplo 8

In [41]:
datos = datos8
BaseDatos(datos)

0. Datos
--------

   a  b  c  d
0  1  1  1  4
1  1  1  2  4
2  1  2  1  4
3  2  2  2  4
4  2  1  1  4
5  2  1  2  4
1. Dependencias funcionales completas
----------------------------------

{ a } -> { d }
{ b } -> { d }
{ c } -> { d }

2. Clave primaria 
------------------

{a, b, c}

3. Cierre Minimo 
--------------

{ a } -> { d }
{ b } -> { d }
{ c } -> { d }

4. Algoritmo 3NF
----------------
   a  d
0  1  4
1  2  4
-----------------------------
   b  d
0  1  4
1  2  4
-----------------------------
   c  d
0  1  4
1  2  4
-----------------------------


# $\hspace{20 cm}$ SevenTenTree